<a href="https://colab.research.google.com/github/meetmehedi/SaveFood/blob/main/Food_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-datasets pillow

In [ ]:
import os
import json
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 6   # Increase for better accuracy
os.makedirs("models", exist_ok=True)

In [ ]:
def preprocess(example):
    image = tf.image.resize(example['image'], (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    label = example['label']
    return image, label


In [ ]:
def prepare_datasets():
    print("📥 Downloading Food-101 dataset...")
    ds_train, ds_info = tfds.load(
        'food101', split='train',
        with_info=True, shuffle_files=True,
        as_supervised=False
    )
    ds_test = tfds.load(
        'food101', split='validation',
        as_supervised=False
    )

    num_classes = ds_info.features['label'].num_classes
    print(f"✅ Found {num_classes} classes.")

    train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train = train.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    val = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    val = val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return train, val, num_classes, ds_info


In [ ]:
def build_model(num_classes):
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base.trainable = False  # Freeze base for faster training

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
train_ds, val_ds, n_classes, ds_info = prepare_datasets()
model = build_model(n_classes)
model.summary()

model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# Save model
model_path = "models/food101_mobilenetv2.h5"
model.save(model_path)

# Save label names
label_names = ds_info.features['label'].names
with open("models/label_names.json", "w") as f:
    json.dump(label_names, f)

print(f"💾 Model saved to {model_path}")
print("💾 Labels saved to models/label_names.json")



📥 Downloading Food-101 dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

 Total params: 2,387,365 (9.11 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/6
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 237s 94ms/step - accuracy: 0.1219 - loss: 4.0371 - val_accuracy: 0.4506 - val_loss: 2.3554
Epoch 2/6
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 243s 102ms/step - accuracy: 0.3859 - loss: 2.5294 - val_accuracy: 0.5196 - val_loss: 1.9589
Epoch 3/6
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 206s 86ms/step - accuracy: 0.4543 - loss: 2.2023 - val_accuracy: 0.5470 - val_loss: 1.8026
Epoch 4/6
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 208s 87ms/step - accuracy: 0.4866 - loss: 2.0525 - val_accuracy: 0.5648 - val_loss: 1.7150
Epoch 5/6
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 246s 103ms/step - accuracy: 0.5089 - loss: 1.9578 - val_accuracy: 0.5745 - val_loss: 1.6600
Epoch 6/6
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 231s 90ms/step - accuracy: 0.5249 - loss: 1.8899 - val_accuracy: 0.5843 - val_loss: 1.6193


💾 Model saved to models/food101_mobilenetv2.h5
💾 Labels saved to models/label_names.json


In [ ]:

from google.colab import files
files.download("models/food101_mobilenetv2.h5")
files.download("models/label_names.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install datasets
!pip install tensorflow
!pip install torch torchvision
!pip install pillow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
import numpy as np
from PIL import Image

In [ ]:
dataset = load_dataset("ethz/food101")
train_ds = dataset['train']
val_ds = dataset['validation']

# Get class names
class_names = train_ds.features['label'].names
num_classes = len(class_names)
print("Number of classes:", num_classes)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

Number of classes: 101


In [ ]:
IMG_SIZE = 224

def preprocess_image(pil_image):
    img = pil_image.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img) / 255.0
    return img_array

def generator(dataset_split, batch_size=32):
    dataset_size = len(dataset_split)
    while True:
        for start in range(0, dataset_size, batch_size):
            end = min(start + batch_size, dataset_size)
            batch_images = []
            batch_labels = []
            for i in range(start, end):
                img = dataset_split[i]['image']
                img_array = preprocess_image(img)
                batch_images.append(img_array)
                label = dataset_split[i]['label']
                batch_labels.append(label)
            yield np.array(batch_images), tf.keras.utils.to_categorical(batch_labels, num_classes=num_classes)


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # freeze base for initial training

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
BATCH_SIZE = 8
STEPS_PER_EPOCH = len(train_ds) // BATCH_SIZE
VALIDATION_STEPS = len(val_ds) // BATCH_SIZE

model.fit(
    generator(train_ds, BATCH_SIZE),
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=generator(val_ds, BATCH_SIZE),
    validation_steps=VALIDATION_STEPS,
    epochs=3  # start with small epochs, increase if needed
)


Epoch 1/3
2313/9468 ━━━━━━━━━━━━━━━━━━━━ 44:44 375ms/step - accuracy: 0.8117 - loss: 1.3295

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (8, 224, 224) + inhomogeneous part.
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/keras/src/trainers/d

In [ ]:
base_model.trainable = True  # unfreeze for fine-tuning
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    generator(train_ds, BATCH_SIZE),
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=generator(val_ds, BATCH_SIZE),
    validation_steps=VALIDATION_STEPS,
    epochs=2  # fine-tuning epochs
)

Epoch 1/2
  90/9468 ━━━━━━━━━━━━━━━━━━━━ 4:02:15 2s/step - accuracy: 0.0000e+00 - loss: 5.6461

KeyboardInterrupt: 

In [ ]:
model.save("/content/food101_mobilenetv2.h5")
print("Model saved as food101_mobilenetv2.h5")

Model saved as food101_mobilenetv2.h5
